## Wahrscheinlichkeitsverteilungen

**Verschiedene Verteilungsfunktionen interaktiv visualisieren**

Verschiedene Funktionen können ausgewählt werden und interaktiv dargestellt erden.

Funktionen:
- Poissonverteilung, [scipy-link](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.poisson.html#scipy.stats.poisson)
- Gamma - Poisson Verteilung, [scipy-link](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.nbinom.html#scipy.stats.nbinom)
- Binomialverteilung, [scipy-link](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.binom.html#scipy.stats.binom)
- Dichtefunktion (Gauss - Funktion), [scipy-link](https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.norm.html)
- Landauverteilung, [scipy-link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.moyal.html)
- Gamma - Verteilung, [scipy-link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html#scipy.stats.gamma)
- Weibullverteilung, [scipy-link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.weibull_min.html#scipy.stats.weibull_min)

In [1]:
# resourcen
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import poisson, gamma, moyal, norm, binom, nbinom, weibull_min

#set backend for interactive toolbar
%matplotlib widget

In [2]:
# während der Berechnung der Verteilungen werden Warnungen ausgegeben.
# Diese werden hier ausgeschalten
import warnings
warnings.filterwarnings('ignore')

In [3]:
# interaktive Schalter
#
# Signalauswahl
distribFuns = ['Poisson-Verteilung', 'Gamma-Poisson-Verteilung',
               'Binomialverteilung','Normalverteilung','Landau-Verteilung',
               'Gamma-Verteilung','Weibull-Verteilung']
distrib_select = widgets.Dropdown(description='', options=distribFuns,
                                  value='Normalverteilung',
                                  layout= widgets.Layout(width='55%',height='auto')
                                 )
#----------------------------------------------------------------------------------
layout= widgets.Layout(width='95%',height='auto')
style = {'description_width': 'initial'}
# Funktionsparameter
# ------------------------------------------------------------------------------------------------------------------------------
#
# Gauss: µ: Erwartungswert, \sigma: Standardabweichung 
mu = widgets.FloatSlider(min=-150, max=150, value=0, continuous_update=False, layout= layout)
sigma = widgets.FloatSlider(min=0.1, max=15, value=1, layout= layout)
#
normLabel_1 = "µ: (Erwartungswert)"
normLabel_2 = "$\sigma$: (Standardabweichung)"

# ------------------------------------------------------------------------------------------------------------------------------
#
# Poisson: Erwartungswert und Varianz µ (geteilt mit Gauss - Erwartungswert), µ > 0 
poisLabel = "µ: (Erwartungswert und Varianz)"
# # ------------------------------------------------------------------------------------------------------------------------------
## Gamma-Poisson => negative Binomialverteilung: 
#                  n: Anzahl Erfolge, alternativ über µ einstellbar n = (µ²/{\sigma}² - µ)
#                  p: Einzelwahrscheinlichkeit , alternativ über µ und sigma einstellbar: p= (µ/{\sigma}²)
#
gammaPoisLabel_1 = "n: (Anzahl der Erfolge n. n > 0 ($mean²\over (var - mean)$)"
gammaPoisLabel_2 = "p: (Wahrscheinlichkeit eines einzelnen Erfolgs. $0 \leq p \leq 1$ (mean/varianz)"

# ----------------------------------------------------------------------------------------------------------------------------
#
# Binomialverteilung: n: Anzahl der Erfolger, p: Einzelwahrscheinlichkeit
# (Schaltflächen mit Gamma-Poisson-Verteilung geteilt)
#
binomLabel_1 = "n: (Anzahl der Erfolge n. n > 0 (mean²/(var - mean))"
binomLabel_2 = "p: (Wahrscheinlichkeit eines einzelnen Erfolgs. $0 \leq p \leq 1$ (mean/varianz)"

# ----------------------------------------------------------------------------------------------------------------------------
#
# Landauverteilung: Lageparameter µ ; Skalenparameter c
# (Schaltflächen mit Normalverteilung und Poisson-Verteilung geteilt)
landauLabel_1 = "µ: (Erwartungswert)"
landauLabel_2 = "$\sigma$: (Standardabweichung)"

# ----------------------------------------------------------------------------------------------------------------------------
#
# Gammaverteilung: inverser Skalenparameter b; Formparameter p
# ----------------------------------------------------------------------------------------------------------------------------
p_gamma = widgets.FloatSlider(min=0, max=150, value=1, step=0.01, continuous_update=False, layout= layout)
#
gammaLabel_1 = "p: Formparater (p>0)"
gammaLabel_2 = "b: (Skalierungsfaktor)"
gammaLabel_3 = "µ: (Mittelwert)"
#
# ----------------------------------------------------------------------------------------------------------------------------
#
# Weibull : - Skalen- und Formparameter:
#           1/lambda > 0
wblLabel_1 = "k: Formparater (k > 0)"
wblLabel_2 = "$\lambda$: (Skalenparameter $\lambda$.$1 \over \lambda > 0$.)"
wblLabel_3 = "µ: (Verschiebeoperator)"
#
# ---------------------------------------------------------------------------------------------------------------------------
# interaktive Schaltflächen in widget 'interakt' zusammengefasst
# - zu jeder gewählten Verteilungsfunktion werden die Schaltflächen angepasst
#
label_1 = widgets.Label(vlaue="", tooltips="")
label_2 = widgets.Label(vlaue="", tooltips="")
label_3 = widgets.Label(vlaue="", tooltips="")

interakt = widgets.VBox([\
                         label_1,
                         widgets.FloatSlider(min=0.1, max=1.0,continuous_update=False),
                         label_2,
                         widgets.FloatSlider(min = 0.0, max=1.1, continuous_update=False),
                         label_3,
                         widgets.FloatSlider(min=0.1,max=1.1, continuous_update=False)
                        ]
                       )
# plot limits
x_range = widgets.IntRangeSlider(min= -200, max=200, value=[-5,5], continuous_update=False)
xrangeBox = widgets.VBox([
    widgets.Label(value="x-range"),
    x_range
])
#
BtnReset = widgets.Button(description="reset", button_style='warning')
# ---------------------------------------------------------------------------------------------------------------------------

In [4]:
def draw_ProbabilityDensity(x, f_distribution, ax):
    ax.clear()
    ax.set_title("Dichtefunktion zur {}".format(distrib_select.value))
    #Verteilung nochmal zeichnen und dann die entsprechenden Informationen eintragen
    if distrib_select.value in ['Poisson-Verteilung', 'Gamma-Poisson-Verteilung', 'Binomialverteilung']:
        ax.bar(x, f_distribution, color='green', width=0.25)
    else:
        ax.plot(x, f_distribution, color='green', lw=2)
        ax.fill_between(x, 0, f_distribution, where =(f_distribution >= 0),\
                   facecolor='green', alpha=0.3)
    # Abszissenbereich entsprechend Bereichsauswahl einstellen
    ax.set_xlim(x_range.value[0], x_range.value[-1])
    ax.set_xlabel("x")
    ax.set_ylabel("Wahrscheinlichkeit")
    #
    # Gitterlinien andeuten
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
               alpha=0.5)

In [5]:
def draw_cumulativeProbability(x, f_distribution, ax):
    ax.clear()
    ax.set_title("kumulative Verteilung zur {}".format(distrib_select.value))
    ax.plot(x, f_distribution, color='C1', lw=2)
    ax.fill_between(x, 0, f_distribution, where =(f_distribution >= 0),\
                   facecolor='C1', alpha=0.3)
    # Abszissenbereich entsprechend Bereichsauswahl einstellen
    ax.set_xlim(x_range.value[0], x_range.value[-1])
    ax.set_xlabel("x")
    ax.set_ylabel("Wahrscheinlichkeit")
    # Gitterlinien andeuten
    ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
               alpha=0.5)

In [6]:
def reset_controls(*btn):
    # Reset Button:
    global interakt
    distrib_select.value = 'Normalverteilung'
    interakt.children[0].value = normLabel_1
    interakt.children[2].value = normLabel_2
    interakt.children[1].disabled = False
    interakt.children[1].min = -150
    interakt.children[1].max = 150
    interakt.children[1].style.handle_color = "green"
    interakt.children[1].value = 0.0
    interakt.children[3].disabled = False
    interakt.children[3].min = 0.01
    interakt.children[3].max = 50
    interakt.children[3].value = 1
    interakt.children[3].step = 0.05
    interakt.children[3].style.handle_color = "green"
    interakt.children[4].disabled = True
    interakt.children[4].value = ""
    interakt.children[4].disabled = True
    interakt.children[4].value = ""
    x_range.value = [-5,5]
    update_view()

In [10]:
# interaktive Visualisierung:
#
#- Figuren Zeichnen:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6,6))

distribType_old = "Poisson-Verteilung"
#
# 1. create a callback which updates the plot when a control-value has changed
def update_view(*args):
    global distribType_old
    global interakt
    
    # selections
    distribType = distrib_select.value
    
    initWidgets = True
    if distribType == distribType_old:
        initWidgets = False
    #-----------------------------------------------------------
    # Datensatz nach der gewählten Verteilungsfunktion erzeugen
    #   
    # abszisse: ppf({...}}) Percent point function (inverse of cdf — percentiles).
    # Dichtefunktion/Wahrscheinlichkeitsfunktion: pdf({...}})/pmf({...}})
    # kummulierte Dichtefunktion: cdf({...}}) Cumulative density function.
    #
    if (distribType == "Normalverteilung"):
        if initWidgets:
            # init intaeraktWidget
            interakt.children[0].value = normLabel_1
            interakt.children[2].value = normLabel_2
            interakt.children[1].disabled = False
            interakt.children[1].min = -150
            interakt.children[1].max = 150
            interakt.children[1].style.handle_color = "green"
            interakt.children[1].value = 0.0
            interakt.children[3].disabled = False
            interakt.children[3].min = 0.01
            interakt.children[3].max = 50
            interakt.children[3].value = 1
            interakt.children[3].step = 0.05
            interakt.children[3].style.handle_color = "green"
            interakt.children[4].disabled = True
            interakt.children[4].value = ""
            interakt.children[4].disabled = True
            interakt.children[4].value = ""
            #
        mu= interakt.children[1].value
        sigma = interakt.children[3].value
        #
        x_dist = np.arange(norm.ppf(0.01, mu, sigma), norm.ppf(0.99, mu, sigma), 1/100)
        distrib_f = norm(mu, sigma)
        #
        dist_prob = distrib_f.pdf(x_dist)
        dist_cdf = distrib_f.cdf(x_dist)
#------------------------------------------------           
    if (distribType == "Poisson-Verteilung"):
        if initWidgets:
            x_range.min = 0
            # init intaeraktWidget
            interakt.children[0].value = poisLabel
            interakt.children[1].disabled = False
            interakt.children[1].min = 0.1
            interakt.children[1].style.handle_color = "green"
            interakt.children[1].value = 3.76
            interakt.children[2].disabled = True
            interakt.children[2].value = ""
            interakt.children[3].style.handle_color = None
            interakt.children[3].disabled = True
            interakt.children[4].disabled = True
            interakt.children[4].value = ""
            interakt.children[4].disabled = True
            interakt.children[5].style.handle_color = None
            interakt.children[5].disabled = True
            #
        mu = interakt.children[1].value
        x_dist = np.arange(poisson.ppf(0.001, mu), poisson.ppf(0.999, mu))
        dist_prob = poisson.pmf(x_dist, mu)
        dist_cdf = poisson.cdf(x_dist, mu)
#--------------------------------------------------------       
    if (distribType == "Gamma-Poisson-Verteilung"):
        if initWidgets:
        # Diagram-Bereich umschalten damit Funktion sichtbar
            x_range.min = 0
            # init intaeraktWidget
            interakt.children[0].disabled = False
            interakt.children[0].value = gammaPoisLabel_1
            interakt.children[1].disabled = False
            interakt.children[1].style.handle_color = "green"
            interakt.children[1].min = 1
            interakt.children[1].max = 500
            interakt.children[1].value = 10
            interakt.children[1].step = 1.0
            interakt.children[2].disabled = False
            interakt.children[2].value = gammaPoisLabel_2
            interakt.children[3].disabled = False
            interakt.children[3].style.handle_color = "green"
            interakt.children[3].min = 0
            interakt.children[3].max = 1
            interakt.children[3].value = 0.5
            interakt.children[3].step = 0.05
            interakt.children[4].disabled = True
            interakt.children[4].value = ""
            interakt.children[5].disabled = True
            interakt.children[5].style.handle_color = None
        n= interakt.children[1].value
        p= interakt.children[3].value
        x_dist = np.arange(nbinom.ppf(0.001, n, p), nbinom.ppf(0.999, n, p))
        #
        distrib_f = nbinom(n, p)
        dist_prob = distrib_f.pmf(x_dist)
        dist_cdf = distrib_f.cdf(x_dist)
#------------------------------------------------------    
    if (distribType == "Binomialverteilung"):
        if initWidgets:
            # Diagram-Bereich umschalten damit Funktion sichtbar
            x_range.min = 0
            # init intaeraktWidget
            interakt.children[0].disabled = False
            interakt.children[0].value = binomLabel_1
            interakt.children[1].disabled = False
            interakt.children[1].style.handle_color = "green"
            interakt.children[1].min = 1
            interakt.children[1].max = 500
            interakt.children[1].value = 10
            interakt.children[1].step = 1.0
            interakt.children[2].disabled = False
            interakt.children[2].value = binomLabel_2
            interakt.children[3].disabled = False
            interakt.children[3].style.handle_color = "green"
            interakt.children[3].min = 0
            interakt.children[3].max = 1
            interakt.children[3].value = 0.5
            interakt.children[3].step = 0.05
            interakt.children[4].disabled = True
            interakt.children[4].value = ""
            interakt.children[5].disabled = True
            interakt.children[5].style.handle_color = None
        #
        n= interakt.children[1].value
        p= interakt.children[3].value
        #
        x_dist = np.arange(binom.ppf(0.01, n, p), binom.ppf(0.99, n, p))
        distrib_f = binom(n, p)
        dist_prob = distrib_f.pmf(x_dist)
        dist_cdf = distrib_f.cdf(x_dist)
#-----------------------------------------------------------        
    if (distribType == "Gamma-Verteilung"):
        if initWidgets:
            # Diagram-Bereich umschalten damit Funktion sichtbar
            x_range.min = 0
            # init intaeraktWidget
            interakt.children[0].disabled = False
            interakt.children[0].value = gammaLabel_1
            interakt.children[1].disabled = False
            interakt.children[1].style.handle_color = "green"
            interakt.children[1].min = 0
            interakt.children[1].max = 150
            interakt.children[1].value = 1
            interakt.children[1].step = 0.05
            interakt.children[2].disabled = False
            interakt.children[2].value = gammaLabel_2
            interakt.children[3].disabled = False
            interakt.children[3].style.handle_color = "green"
            interakt.children[3].min = 0
            interakt.children[3].max = 15
            interakt.children[3].value = 0.05
            interakt.children[3].step = 0.05
            interakt.children[4].disabled = False
            interakt.children[4].value = gammaLabel_3
            interakt.children[5].disabled = False
            interakt.children[5].style.handle_color = "green"
            interakt.children[5].min = 0
            interakt.children[5].value = 0
            interakt.children[5].step = 1.0
            interakt.children[5].max = 150
        #
        p = interakt.children[1].value
        sigma = interakt.children[3].value
        mu = interakt.children[5].value
        x_dist = np.arange(gamma.ppf(0.001, p, loc=mu), gamma.ppf(0.999, p, loc=mu), 1/100)
        #
        distrib_f = gamma(p, loc= mu, scale= sigma)
        dist_prob = distrib_f.pdf(x_dist)
        dist_cdf = distrib_f.cdf(x_dist)
#----------------------------------------------------------

    if (distribType == "Landau-Verteilung"):
        if initWidgets:
            # Diagram-Bereich umschalten damit Funktion sichtbar        
            x_range.min = -10
            # init intaeraktWidget
            interakt.children[0].disabled = False
            interakt.children[0].value = landauLabel_1
            interakt.children[1].disabled = False
            interakt.children[1].style.handle_color = "green"
            interakt.children[1].min = -150
            interakt.children[1].max = 150
            interakt.children[1].value = 1
            interakt.children[1].step = 0.1
            interakt.children[2].disabled = False
            interakt.children[2].value = landauLabel_2
            interakt.children[3].disabled = False
            interakt.children[3].style.handle_color = "green"
            interakt.children[3].min = 0
            interakt.children[3].max = 15
            interakt.children[3].step = 0.05
            interakt.children[4].disabled = True
            interakt.children[4].value = ""
            interakt.children[5].disabled = True
            interakt.children[5].style.handle_color = None
            interakt.children[5].min = 0
            interakt.children[5].value = 0
        #
        mu = interakt.children[1].value
        sigma = interakt.children[3].value
        x_dist = np.arange(moyal.ppf(0.001), moyal.ppf(0.999), 1/100)
        #
        dist_prob = moyal.pdf(x_dist,loc=mu, scale=sigma)
        dist_cdf = moyal.cdf(x_dist,loc=mu, scale=sigma)
#--------------------------------------------------------
    if (distribType == "Weibull-Verteilung"):
        if initWidgets:
            # init intaeraktWidget
            interakt.children[1].disabled = False
            interakt.children[1].style.handle_color = "green"
            interakt.children[1].min = 0.1
            interakt.children[1].max = 15
            interakt.children[1].value = 1.69
            interakt.children[1].step = 0.05
            interakt.children[2].disabled = False
            interakt.children[2].value = wblLabel_2
            interakt.children[3].disabled = False
            interakt.children[3].style.handle_color = "green"
            interakt.children[3].min = 0.01
            interakt.children[3].max = 100
            interakt.children[3].value = 1
            interakt.children[3].step = 0.05
            interakt.children[4].disabled = False
            interakt.children[4].value = wblLabel_3
            interakt.children[5].disabled = False
            interakt.children[5].style.handle_color = "green"
            interakt.children[5].min = 0
            interakt.children[5].value = 0
            interakt.children[5].step = 0.5
            interakt.children[5].max = 150
        #
        form_wbl = interakt.children[1].value
        lambda_wbl = interakt.children[3].value
        mu = interakt.children[5].value
        x_dist = np.linspace(weibull_min.ppf(0.01, form_wbl, loc= mu), weibull_min.ppf(0.99, form_wbl, loc=mu),100)
        distrib_f = weibull_min(form_wbl, scale=lambda_wbl, loc= mu)
        #
        dist_prob = distrib_f.pdf(x_dist)
        dist_cdf = distrib_f.cdf(x_dist)
#---------------------------------------------------------------------------
# diagramme aktualisieren
#
#
    draw_ProbabilityDensity(x_dist, dist_prob, axes[0])
    draw_cumulativeProbability(x_dist, dist_cdf, axes[1])
    
    distribType_old = distribType
    #
    fig.tight_layout()
    #  
#--------------------------------------------------------
# 2. die Callback-Funktion mit der Funktion 'observe' den Steuerelementen zuweisen
    
distrib_select.observe(update_view, 'value')
x_range.observe(update_view, 'value')
interakt.children[1].observe(update_view, 'value')
interakt.children[3].observe(update_view, 'value')
interakt.children[5].observe(update_view, 'value')
BtnReset.on_click(reset_controls)
#--------------------------------------------------------
# Anwendung starten
#
# Diagramme einmal zeichnen
update_view()
#
# mit 'widgets.VBox / .HBox' die Steuerelemente arangieren
widgets.VBox([
    widgets.HBox([
        widgets.VBox([
            widgets.Label(value="Funktionsauswahl:", layout= widgets.Layout(width='55%',height='auto')),
            distrib_select,
            widgets.HTML(value="<hr>")
            ],
            layout={'width': '30%'}
        ),
        widgets.VBox([
            interakt,
            xrangeBox,
        ],
            layout={'width': '50%'}
        )
        ]),
    widgets.HTML(value="<hr>"),
    BtnReset
])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Erläuterungen zur interaktiven Nutzung**
<br>
<br>
Funktionsauswahl: $\quad$ Funktion auswaehlen. Bei Funktionsaenderung werden default-Parameter eingestellt.<br>
Schieberegler: $\qquad$ Einstellungen zur gewählten Funktion. Parameter werden entsprechend der gewählten Funktion bezeichnet und freigeschalten.<br>


Copyright © 2020 IUBH Internationale Hochschule